In [10]:
#!apt-get install -y libgl1-mesa-dev
!pip3 install --upgrade pip
!pip3 install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio==0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
ERROR: Could not find a version that satisfies the requirement torch==1.8.1+cu111 (from versions: 0.1.2, 0.1.2.post1, 0.1.2.post2, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0)
ERROR: No matching distribution found for torch==1.8.1+cu111


In [9]:
!pip3 install numpy
!pip3 install pandas
!pip3 install multiprocessing
!pip3 install botocore
!pip3 install boto3
!pip3 install scikit-build
!pip3 install cmake
!pip3 install tqdm
!pip3 install opencv-python

  Using cached multiprocessing-2.6.2.1.tar.gz (108 kB)
    ERROR: Command errored out with exit status 1:
     command: /Users/naoya/opt/anaconda3/bin/python -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/_x/_vywjr3167l7ptvbm__phtm40000gn/T/pip-install-6dailnnd/multiprocessing_82e147e6d26645cea8ee5845d062c818/setup.py'"'"'; __file__='"'"'/private/var/folders/_x/_vywjr3167l7ptvbm__phtm40000gn/T/pip-install-6dailnnd/multiprocessing_82e147e6d26645cea8ee5845d062c818/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base /private/var/folders/_x/_vywjr3167l7ptvbm__phtm40000gn/T/pip-pip-egg-info-vq16opmc
         cwd: /private/var/folders/_x/_vywjr3167l7ptvbm__phtm40000gn/T/pip-install-6dailnnd/multiprocessing_82e147e6d2

In [ ]:
#確認
import os
os.environ.pop('CUDA_LAUNCH_BLOCKING', None)
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
import cv2
import logging
import pandas as pd
import logging
import os
from multiprocessing import Pool, Manager
import functools
import tqdm

ModuleNotFoundError: No module named 'cv2'

In [13]:
#make model
from vision.ssd.mobilenetv3_ssd_lite import create_mobilenetv3_small_ssd_lite
from vision.ssd.config import mobilenetv1_ssd_config
create_net = lambda num: create_mobilenetv3_small_ssd_lite(num, onnx_compatible=True)
config = mobilenetv1_ssd_config

ModuleNotFoundError: No module named 'torch'

In [ ]:
#prepare tranform
from vision.ssd.data_preprocessing import TrainAugmentation, TestTransform
from vision.ssd.ssd import MatchPrior
train_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
target_transform = MatchPrior(config.priors, config.center_variance, config.size_variance, 0.5)
test_transform = TrainAugmentation(config.image_size, config.image_mean, config.image_std)
logging.info("Prepare training datasets.")

In [ ]:
#optunaでカリカリにチューニングする
Param = {
    "batch_size" : 8,
    "base_net_lr" : 0.001, #tune
    "num_epochs" : 200,
    "validation_epochs" : 5,
    "t_max" : 100, #tune
    "lr" : 0.01, #tune
    "freeze_base_net" : False, #ファインチューニングするときはTrue,
    "freeze_net" : False,
    "num_workers" : 2, #並列処理 windows対応してないです…
    "momentum" : 0.9, #tune
    "weight_decay" : 5e-4, #tune
    "t_max" : 100, #tune
    "debug_steps" : 100,
}

In [ ]:
dataset_path = "./all_images"
check_point_path = "./models/gaku_cornv3"
os.makedirs(check_point_path, exist_ok=True)

In [9]:
from vision.utils.misc import str2bool, Timer, freeze_net_layers, store_labels
from torch.utils.data import DataLoader, ConcatDataset
#TODO データセットのライブラリ作る、transformも -> OpenImagesDatasetをたたき台にする 最終的にsub-{}-annotation-bboxに統合する
from vision.datasets.open_images import OpenImagesDataset

datasets = []
#dawnsampling
dataset = OpenImagesDataset(dataset_path,transform=train_transform, target_transform=target_transform, dataset_type="train", balance_data=False)
label_file = os.path.join(check_point_path, "open-images-model-labels.txt")
if os.path.exists(label_file):
    os.remove(label_file)
store_labels(label_file, dataset.class_names)

logging.info(dataset)
num_classes = len(dataset.class_names)
print("class name : ", dataset.class_names)

datasets.append(dataset)

num_classes = len(dataset.class_names)
print(label_file) #models/open-images-model-labels.txtに対象載ってる
train_dataset = ConcatDataset(datasets)
print(len(train_dataset))
train_loader = DataLoader(train_dataset, Param["batch_size"],num_workers=Param["num_workers"],shuffle=True)

nan sum :  -35
class name :  ['BACKGROUND', 'can', 'trafficcone']
./models/gaku_cornv1/open-images-model-labels.txt
317


In [10]:
logging.info("Prepare Validation datasets.")
val_dataset = OpenImagesDataset(dataset_path,transform=test_transform, target_transform=target_transform, dataset_type="test")
logging.info(val_dataset)
logging.info("validation dataset size: {}".format(len(val_dataset)))
val_loader = DataLoader(val_dataset,  Param["batch_size"],num_workers=Param["num_workers"],shuffle=False)

nan sum :  -14


In [11]:
#if fine tuning, change this cell
import itertools

logging.info("Build network.")
net = create_net(num_classes)
min_loss = -10000.0
last_epoch = -1 #pretrain
#model_path = "./models/gaku_cornv1/mbv3-v3-Epoch-15-Loss-6.931226122763849.pth" #no
model_path = "no"
if model_path != "no":
    net.init_from_pretrained_ssd(model_path)
base_net_lr = Param["base_net_lr"]
extra_layers_lr = Param["lr"]
if Param["freeze_base_net"] == True:
    logging.info("Freeze base net.")
    freeze_net_layers(net.base_net)
    params = itertools.chain(net.source_layer_add_ons.parameters(), net.extras.parameters(),
                            net.regression_headers.parameters(), net.classification_headers.parameters())
    params = [
        {'params': itertools.chain(
            net.source_layer_add_ons.parameters(),
            net.extras.parameters()
        ), 'lr': extra_layers_lr},
        {'params': itertools.chain(
            net.regression_headers.parameters(),
            net.classification_headers.parameters()
        )}
    ]
elif Param["freeze_base_net"] == True:
    freeze_net_layers(net.base_net)
    freeze_net_layers(net.source_layer_add_ons)
    freeze_net_layers(net.extras)
    params = itertools.chain(net.regression_headers.parameters(), net.classification_headers.parameters())
    logging.info("Freeze all the layers except prediction heads.")
else:
    params = [
        {'params': net.base_net.parameters(), 'lr': base_net_lr},
        {'params': itertools.chain(
            net.source_layer_add_ons.parameters(),
            net.extras.parameters()
        ), 'lr': extra_layers_lr},
        {'params': itertools.chain(
            net.regression_headers.parameters(),
            net.classification_headers.parameters()
        )}
    ]

In [12]:
#pre-train使うなら argsは頑張る
#if fine tuning, change this cell
"""
if args.resume:
    logging.info(f"Resume from the model {args.resume}")
    net.load(args.resume)
elif args.base_net:
    logging.info(f"Init from base net {args.base_net}")
    net.init_from_base_net(args.base_net)
elif args.pretrained_ssd:
    logging.info(f"Init from pretrained ssd {args.pretrained_ssd}")
    net.init_from_pretrained_ssd(args.pretrained_ssd)
logging.info(f'Took {timer.end("Load Model"):.2f} seconds to load the model.')
"""

'\nif args.resume:\n    logging.info(f"Resume from the model {args.resume}")\n    net.load(args.resume)\nelif args.base_net:\n    logging.info(f"Init from base net {args.base_net}")\n    net.init_from_base_net(args.base_net)\nelif args.pretrained_ssd:\n    logging.info(f"Init from pretrained ssd {args.pretrained_ssd}")\n    net.init_from_pretrained_ssd(args.pretrained_ssd)\nlogging.info(f\'Took {timer.end("Load Model"):.2f} seconds to load the model.\')\n'

In [13]:
net.to(DEVICE)

SSD(
  (base_net): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): hswish()
    (3): Block(
      (se): SeModule(
        (se): Sequential(
          (0): AdaptiveAvgPool2d(output_size=1)
          (1): Conv2d(16, 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (2): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): ReLU(inplace=True)
          (4): Conv2d(4, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (5): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (6): hsigmoid()
        )
      )
      (conv1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (nolinear1): ReLU(inplace=True)
      (conv2): Conv2d(16, 16, kernel_

In [14]:
from vision.nn.multibox_loss import MultiboxLoss
from torch.optim.lr_scheduler import CosineAnnealingLR, MultiStepLR

criterion = MultiboxLoss(config.priors, iou_threshold=0.5, neg_pos_ratio=3,
                             center_variance=0.1, size_variance=0.2, device=DEVICE)
optimizer = torch.optim.SGD(params, lr=Param["lr"], momentum=Param["momentum"],
                            weight_decay=Param["weight_decay"])
scheduler = CosineAnnealingLR(optimizer, Param["t_max"], last_epoch=-1)

In [15]:
"""
for _ in range(last_epoch):
    print("skip")
    scheduler.step()
"""

skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip
skip


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


In [16]:
def train(loader, net, criterion, optimizer, device, debug_steps=100, epoch=-1):
    net.train(True)
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    for i, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        confidence, locations = net(images)
        #print(boxes.shape, labels.shape, confidence.shape, locations.shape)
        regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)  # TODO CHANGE BOXES
        loss = regression_loss + classification_loss
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
        if i and i % debug_steps == 0:
            avg_loss = running_loss / debug_steps
            avg_reg_loss = running_regression_loss / debug_steps
            avg_clf_loss = running_classification_loss / debug_steps
            
            logging.info(
                f"Epoch: {epoch}, Step: {i}, " +
                f"Average Loss: {avg_loss:.4f}, " +
                f"Average Regression Loss {avg_reg_loss:.4f}, " +
                f"Average Classification Loss: {avg_clf_loss:.4f}"
            )
            """
            print("epoch: ", epoch)
            print("Average Loss: ", avg_loss)
            print("Average Regression Loss: ", avg_reg_loss)
            print("Average Classification Loss: ", avg_clf_loss)
            """
            running_loss = 0.0
            running_regression_loss = 0.0
            running_classification_loss = 0.0

In [17]:
def test(loader, net, criterion, device):
    print("start test")
    net.eval()
    running_loss = 0.0
    running_regression_loss = 0.0
    running_classification_loss = 0.0
    num = 0
    for _, data in enumerate(loader):
        images, boxes, labels = data
        images = images.to(device)
        boxes = boxes.to(device)
        labels = labels.to(device)
        num += 1

        with torch.no_grad():
            confidence, locations = net(images)
            #print("validation", boxes.shape, labels.shape, confidence.shape, locations.shape)
            regression_loss, classification_loss = criterion(confidence, locations, labels, boxes)
            loss = regression_loss + classification_loss

        running_loss += loss.item()
        running_regression_loss += regression_loss.item()
        running_classification_loss += classification_loss.item()
    return running_loss / num, running_regression_loss / num, running_classification_loss / num

In [18]:
for epoch in range(last_epoch + 1, Param["num_epochs"]):
    print("from", last_epoch + 1, " now : ", epoch, ' last : ', Param["num_epochs"])
    scheduler.step()
    train(train_loader, net, criterion, optimizer,
            device=DEVICE, debug_steps=Param["debug_steps"], epoch=epoch)
    
    if epoch % Param["validation_epochs"] == 0 or epoch == Param["num_epochs"] - 1:
        val_loss, val_regression_loss, val_classification_loss = test(val_loader, net, criterion, device=DEVICE)
        logging.info(
            f"Epoch: {epoch}, " +
            f"Validation Loss: {val_loss:.4f}, " +
            f"Validation Regression Loss {val_regression_loss:.4f}, " +
            f"Validation Classification Loss: {val_classification_loss:.4f}"
        )
        model_path = os.path.join(check_point_path, "mbv3-Epoch-" + str(epoch).zfill(3) + "-Loss- " + val_loss + ".pth")
        net.save(model_path)
        logging.info(f"Saved model {model_path}")

from 16  now :  16  last :  200


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/usr/local/lib/python3.6/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


from 16  now :  17  last :  200


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


from 16  now :  18  last :  200


...

/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


from 16  now :  199  last :  200


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)


start test


/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
/mnt/carryber/pytorch-ssd/vision/transforms/transforms.py:247: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  mode = random.choice(self.sample_options)
